# Table of Contents
 <p><div class="lev1 toc-item"><a href="#Access-all-HRV-records-of-a-single-user-from-spikee-data-base" data-toc-modified-id="Access-all-HRV-records-of-a-single-user-from-spikee-data-base-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Access all HRV records of a single user from spikee data base</a></div><div class="lev2 toc-item"><a href="#Prelimnary-Work" data-toc-modified-id="Prelimnary-Work-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Prelimnary Work</a></div><div class="lev3 toc-item"><a href="#Create-PostgreSQL-DB-for-experiments-by-copying-spikee-Development-DB" data-toc-modified-id="Create-PostgreSQL-DB-for-experiments-by-copying-spikee-Development-DB-1.1.1"><span class="toc-item-num">1.1.1&nbsp;&nbsp;</span>Create PostgreSQL DB for experiments by copying spikee Development DB</a></div><div class="lev2 toc-item"><a href="#Connect-do-Database" data-toc-modified-id="Connect-do-Database-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Connect do Database</a></div><div class="lev2 toc-item"><a href="#SQL-Query-on-joined-tables" data-toc-modified-id="SQL-Query-on-joined-tables-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>SQL Query on joined tables</a></div><div class="lev2 toc-item"><a href="#Calculate-Mean-HR-for-each-Measurement" data-toc-modified-id="Calculate-Mean-HR-for-each-Measurement-1.4"><span class="toc-item-num">1.4&nbsp;&nbsp;</span>Calculate Mean HR for each Measurement</a></div>

# Access all HRV records of a single user from spikee data base
* Author: Johannes Maucher
* Last Update: 2017-05-04
* References:
    * Blog Getting Started with RPostgreSQL: [https://www.r-bloggers.com/getting-started-with-postgresql-in-r/](https://www.r-bloggers.com/getting-started-with-postgresql-in-r/).
* Topics of this notebook
    * Access data from PostgreSQL Database (local oder remote)
    * Join data from different tables

## Prelimnary Work

### Create PostgreSQL DB for experiments by copying spikee Development DB

1. In pgAdmin III right-click on the database (hrv_web) that shall be copied
2. From the context menu choose *Sicherung*
3. Choose directory and file for the database dump
4. In pgAdmin III create a new database, e.g. *experiments*
5. Right-click on the new database and choose *Wiederherstellen (Restore)* from the context menu.
6. Choose the database dump file, which has been created in step 2.

In the code below, we assume that the new database has
* name: experiments
* user: postgres
* pw: hrvanalyzer



In [295]:
options(repos=structure(c(CRAN="http://cloud.r-project.org/")))

In [296]:
library(RPostgreSQL)

## Connect do Database

In [101]:
pw<-{
    "hrvanalyzer"
}
drv<-dbDriver("PostgreSQL")
con <- dbConnect(drv,dbname="hrv_web",host="localhost",port=5432,user="hrv_testuser",password=pw)

In [273]:
pw<-{
    "hrvanalyzer"
}
drv<-dbDriver("PostgreSQL")
conExp <- dbConnect(drv,dbname="experiments",host="localhost",port=5432,user="postgres",password=pw)

In [155]:
hostAWS<-"pg-db-instance-5.cjqqslz7awng.eu-central-1.rds.amazonaws.com"
userAWS<-"hrv_testuser"
pwAWS<-{
    "elephant_runner_1215"
}

In [211]:
conAWS <- dbConnect(drv,dbname="hrv_web",host=hostAWS,port=5432,user=userAWS,password=pwAWS)

In [274]:
print(dbExistsTable(conExp,"profiles_webuser"))

[1] TRUE


In [275]:
df_postgres <- dbGetQuery(conExp, "SELECT id,username,last_name,last_login from profiles_webuser")

In [251]:
df_postgres[order(df_postgres$last_login),]

,id,username,last_name,last_login
1,2,monikamunter,Munter,2015-09-19 14:27:29
2,8,dheinzelmann,,2015-10-04 21:11:29
3,6,maucher,,2015-11-11 19:52:52
7,7,johannes.maucher,,2016-05-18 07:12:29
4,3,timbuktu,Buktu,2016-06-26 07:17:26
6,5,finnland,Land,2016-07-11 12:07:30
8,4,margaquark,Quark,2016-07-14 16:38:23
5,1,lassereden,Reden,2017-05-04 07:23:01


In [276]:
print(dbExistsTable(conExp,"records_data"))

[1] TRUE


## SQL Query on joined tables
General introduction in PostgreSQL see (https://www.postgresql.org/docs/8.4/static/queries-table-expressions.html)[https://www.postgresql.org/docs/8.4/static/queries-table-expressions.html]

Query all morning HRV-Measurements (entire time-series data, not only calculated HRV-Parameters):

In [277]:
mid<-1   #Choose ID of user
mystr2=sprintf("SELECT * FROM records_record INNER JOIN 
records_data ON records_record.id = records_data.record_id 
WHERE records_record.user_id=%d AND records_record.record_type_id=1",mid)
morningMeas<-dbGetQuery(conExp, mystr2)

In [278]:
morningMeas<-morningMeas[order(morningMeas$start),]
dim(morningMeas)

[1] 5776   51

In [279]:
head(morningMeas[c("id","start","timestamp","hr","rr")])
tail(morningMeas[c("id","start","timestamp","hr","rr")])

,id,start,timestamp,hr,rr
1808,3377,2017-04-18 05:13:38,2017-04-18 05:13:39,42,1414
1809,3377,2017-04-18 05:13:38,2017-04-18 05:13:41,36,1653
1810,3377,2017-04-18 05:13:38,2017-04-18 05:13:42,42,1425
1811,3377,2017-04-18 05:13:38,2017-04-18 05:13:43,50,1187
1812,3377,2017-04-18 05:13:38,2017-04-18 05:13:45,44,1345
1813,3377,2017-04-18 05:13:38,2017-04-18 05:13:46,39,1517


,id,start,timestamp,hr,rr
1802,3376,2017-05-04 05:13:33,2017-05-04 05:22:26,39,1523
1803,3376,2017-05-04 05:13:33,2017-05-04 05:22:27,45,1312
1804,3376,2017-05-04 05:13:33,2017-05-04 05:22:29,46,1296
1805,3376,2017-05-04 05:13:33,2017-05-04 05:22:30,37,1604
1806,3376,2017-05-04 05:13:33,2017-05-04 05:22:32,39,1502
1807,3376,2017-05-04 05:13:33,2017-05-04 05:22:33,39,1538


Query for each ID the corresponding date and start time of measurement:

In [280]:
mystr3=sprintf("SELECT id,start FROM records_record WHERE records_record.user_id = %d AND records_record.record_type_id=1 ORDER BY start",mid)
startTimes<-dbGetQuery(conExp, mystr3)

In [281]:
head(startTimes,40)
dim(startTimes)
#startTimes[["id"]]

id,start
3377,2017-04-18 05:13:38
3379,2017-04-19 05:22:17
3381,2017-04-20 05:25:15
3383,2017-04-21 05:22:12
3385,2017-04-22 05:56:47
3387,2017-04-23 06:08:26
3389,2017-04-24 05:12:14
3391,2017-04-25 05:12:48
3393,2017-04-26 05:12:54
3395,2017-04-27 05:13:03


[1] 16  2

## Calculate Mean HR for each Measurement

In [282]:
meanHR<-aggregate(morningMeas$hr, by=list(Id=morningMeas$id,date=morningMeas$start),FUN=mean, na.rm=TRUE)

In [283]:
meanHR<-meanHR[order(meanHR$date),]

In [284]:
head(meanHR,30)

Id,date,x
3377,2017-04-18 05:13:38,42.89005
3379,2017-04-19 05:22:17,42.96065
3381,2017-04-20 05:25:15,43.69697
3383,2017-04-21 05:22:12,45.13528
3385,2017-04-22 05:56:47,45.32967
3387,2017-04-23 06:08:26,44.92188
3389,2017-04-24 05:12:14,40.29766
3391,2017-04-25 05:12:48,43.05085
3393,2017-04-26 05:12:54,45.61499
3395,2017-04-27 05:13:03,42.90291


In [285]:
single<-morningMeas[morningMeas$id==3376,c("timestamp","hr")]
single<-single[order(single$timestamp),]
tail(single)

,timestamp,hr
1802,2017-05-04 05:22:26,39
1803,2017-05-04 05:22:27,45
1804,2017-05-04 05:22:29,46
1805,2017-05-04 05:22:30,37
1806,2017-05-04 05:22:32,39
1807,2017-05-04 05:22:33,39


In [286]:
dim(meanHR)
morningMeas[morningMeas$id==3376,"hr"]
length(morningMeas[morningMeas$id==3376,"hr"])

[1] 16  3

[1] 43 41 48 43 49 48 49 42 46 47 42 44 46 42 43 46 42 43 45 41 42 46 39 40 44
 [26] 38 39 44 40 43 43 38 47 43 42 42 48 40 48 45 41 48 46 43 46 50 45 42 48 45
 [51] 44 49 47 44 48 46 41 45 42 42 47 43 44 49 46 43 48 44 45 49 43 47 48 44 48
 [76] 48 43 48 46 43 50 45 43 48 43 45 45 41 44 47 44 44 48 42 40 45 42 41 43 50
[101] 48 42 47 41 43 47 44 50 45 50 49 50 51 48 52 50 45 49 50 46 46 50 48 45 46
[126] 40 41 44 41 41 47 43 38 38 37 41 45 37 38 44 39 39 45 38 40 43 36 42 44 39
[151] 46 46 42 47 48 43 49 51 46 47 50 46 45 51 48 45 51 49 43 46 42 39 42 45 41
[176] 42 49 47 38 41 42 41 46 39 41 48 46 47 53 49 47 53 54 51 53 54 48 46 51 49
[201] 49 54 56 52 49 52 46 44 49 47 45 48 50 42 46 45 42 41 44 49 41 39 45 42 41
[226] 40 38 42 47 40 40 49 46 42 51 35 38 46 41 44 51 48 45 51 52 50 48 53 51 46
[251] 48 52 49 45 47 50 47 45 49 48 41 40 46 41 41 45 49 41 43 45 41 37 44 44 42
[276] 39 35 37 47 45 42 47 39 39 46 41 41 47 43 44 49 46 52 51 46 45 47 40 40 44
[301] 45 42 41 42 49 45 37 42 41 41 38 40 38 42 46 42 41 47 40 39 48 44 40 44 36
[326] 43 44 40 43 44 41 41 48 47 41 40 46 44 38 39 41 36 38 41 40 41 45 40 39 42
[351] 44 40 39 46 45 39 48 37 38 48 44 42 48 48 44 43 49 46 42 43 46 42 41 43 38
[376] 39 40 47 44 40 44 42 35 40 41 38 44 44 39 45 40 38 46 40 39 45 46 37 39 39

[1] 400

In [287]:
length(unique(meanHR[["Id"]]))

[1] 16

In [288]:
all<-c()
last100<-c()
last200<-c()
last300<-c()
minAll<-c()
maxAll<-c()
sdAll<-c()
q25All<-c()
q50All<-c()
q75All<-c()
#startdate<-c()
allids<-startTimes[["id"]]
for (idx in allids){
    single<-morningMeas[morningMeas$id==idx,c("timestamp","hr")]
    single<-single[order(single$timestamp),]
    hrall<-mean(single$hr,na.rm=TRUE)
    std<-sd(single$hr,na.rm=TRUE)
    hr100<-mean(tail(single$hr,100),na.rm=TRUE)
    hr200<-mean(tail(single$hr,200),na.rm=TRUE)
    hr300<-mean(tail(single$hr,300),na.rm=TRUE)
    all<-c(all,hrall)
    q<-quantile(single$hr,na.rm=TRUE)
    last100<-c(last100,hr100)
    last200<-c(last200,hr200)
    last300<-c(last300,hr300)
    minAll<-c(minAll,q["0%"])
    maxAll<-c(maxAll,q["100%"])
    sdAll<-c(sdAll,std)
    q25All<-c(q25All,q["25%"])
    q50All<-c(q50All,q["50%"])
    q75All<-c(q75All,q["75%"])
}

In [289]:
varLengthHR<-NULL
varLengthHR<-data.frame(id=allids,date=startTimes[["start"]],Mcomplete=all,Stdcomplete=sdAll,Mlast300=last300,Mlast200=last200,Mlast100=last100,
                       MinCompl=minAll,Q25Compl=q25All,Q50Compl=q50All,Q75Compl=q75All,MaxCompl=maxAll)
rownames(varLengthHR)<-varLengthHR$id
varLengthHR$id<-NULL

In [290]:
tail(varLengthHR,30)

,date,Mcomplete,Stdcomplete,Mlast300,Mlast200,Mlast100,MinCompl,Q25Compl,Q50Compl,Q75Compl,MaxCompl
3377,2017-04-18 05:13:38,42.89005,4.383632,43.49333,41.940,42.88,34,40.00,42,45.0,58
3379,2017-04-19 05:22:17,42.96065,4.853379,42.83000,42.545,41.22,31,39.00,43,47.0,56
3381,2017-04-20 05:25:15,43.69697,4.705778,43.69697,43.315,42.84,33,40.00,44,47.0,58
3383,2017-04-21 05:22:12,45.13528,5.638041,44.98000,45.080,44.10,32,41.00,45,49.0,62
3385,2017-04-22 05:56:47,45.32967,4.717484,44.90000,44.345,43.75,35,42.00,45,49.0,58
3387,2017-04-23 06:08:26,44.92188,4.990879,44.67667,44.890,43.95,31,41.75,45,49.0,57
3389,2017-04-24 05:12:14,40.29766,3.925698,40.29766,39.855,39.55,32,37.00,40,43.0,49
3391,2017-04-25 05:12:48,43.05085,5.095095,43.05085,42.335,41.99,30,40.00,42,47.0,59
3393,2017-04-26 05:12:54,45.61499,4.712356,45.88667,45.230,45.24,32,42.00,46,49.0,56
3395,2017-04-27 05:13:03,42.90291,4.539129,42.22667,41.925,42.31,27,40.00,42,45.0,57


In [291]:
head(meanHR,20)

Id,date,x
3377,2017-04-18 05:13:38,42.89005
3379,2017-04-19 05:22:17,42.96065
3381,2017-04-20 05:25:15,43.69697
3383,2017-04-21 05:22:12,45.13528
3385,2017-04-22 05:56:47,45.32967
3387,2017-04-23 06:08:26,44.92188
3389,2017-04-24 05:12:14,40.29766
3391,2017-04-25 05:12:48,43.05085
3393,2017-04-26 05:12:54,45.61499
3395,2017-04-27 05:13:03,42.90291


In [292]:
write.table(varLengthHR,"../data/maucherExperimentVarHR-2017-05-04.csv",sep = ",")

In [85]:
require(rJava)
require(xlsx)
write.xlsx(hrvData,"../data/maucherHRV-2017-05-02.xlsx")

Loading required package: rJava
Loading required package: xlsx
Loading required package: rJava


ERROR: Error in eval(expr, envir, enclos): konnte Funktion "write.xlsx" nicht finden


In [294]:
dbDisconnect(conAWS)
dbDisconnect(conExp)
dbDisconnect(con)

[1] TRUE

[1] TRUE

[1] TRUE

In [ ]:
library(ggplot2)

In [ ]:
ggplot(data = hrvData) + geom_density(mapping=aes(x=rmssd),fill='red',alpha=.4)

In [ ]:
start<-1
end<-780
end<-min(end,dim(hrvData)[1])
xt<-start:end

In [ ]:
ggplot(data = hrvData[xt,]) + geom_line(mapping=aes(x=xt,y=rmssd),color="red")

In [ ]:
ggplot(data = hrvData[xt,]) + geom_line(mapping=aes(x=xt,y=log(rmssd)),color="red")

In [ ]:
ggplot(data = hrvData[xt,]) + geom_line(mapping=aes(x=xt,y=filter(log(rmssd),c(1,1,1,1,1,1,1)/7,circular=TRUE)),color="red")

In [ ]:
ggplot(data = hrvData[xt,]) + geom_point(mapping=aes(x=xt,y=rmssd),color="red") + geom_smooth(mapping=aes(x=xt,y=rmssd))

In [ ]:
ggplot(data = hrvData[xt,]) + geom_point(mapping=aes(x=xt,y=power_hf),color="blue",alpha=.3) + 
geom_smooth(mapping=aes(x=xt,y=power_hf)) +
geom_point(mapping=aes(x=xt,y=power_lf),color="red",alpha=.3) + 
geom_smooth(mapping=aes(x=xt,y=power_lf),color="red")

In [ ]:
hrvData[start:end,]